In [8]:
import pandas as pd
import nltk
#import zhon
import re
import plac
import spacy
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

In [ ]:
df = pd.DataFrame(columns=["en","cn","pos"])

In [ ]:
from zhon import hanzi

In [ ]:
count = 0
df = pd.DataFrame(columns=["en","cn","pos"])

with open("./cn_en_phrases/cn_sentences.txt") as f:

    for line in f:
        splitat = 0
        num = ""
        cn = ""
        en = ""
        vp = False

        numarr = re.findall(r"\d[\d,.:：]*", line)
                    
        if numarr:
            for num in numarr:
                line = line.replace(str(num), "*")
                vp = True
        
        for i in range(len(line)):
            if re.findall('[{}]'.format(zhon.hanzi.characters), line[i]):
                splitat = i
                break
                        
        en = line[:splitat-1].strip()
        cn = line[splitat-1:].strip()
        
        df.loc[count, 'en'] = en
        df.loc[count, 'cn'] = cn
        df.loc[count, 'pos'] = "vp" if vp else "sent"
        count += 1


In [ ]:
df.shape

In [ ]:
with open("./cn_en_phrases/cn_words.txt") as f:

    for line in f:
        splitat = 0
        num = ""
        cn = ""
        en = ""
        """
        numarr = re.findall(r"\d[\d,.:：]*", line)
                    
        if numarr:
            for num in numarr:
                line = line.replace(str(num), "*")
        
        """
        
        for i in range(len(line)):
            if re.findall('[{}]'.format(zhon.hanzi.characters), line[i]):
                splitat = i
                break
                        
        en = line[:splitat-1].strip().rstrip(".").lower()
        cn = line[splitat-1:].strip().rstrip("。")
        
        df.loc[count, 'en'] = en
        df.loc[count, 'cn'] = cn
        df.loc[count, 'pos'] = "np"
        #print(en)
        #print(cn)
        count += 1


In [ ]:
df.head(50)

In [ ]:
df.shape

In [ ]:
df.to_csv("cn_en_added.csv")

---
# Rerun

In [ ]:
df = pd.read_csv("cn_en_added.csv")
master = pd.read_csv("master0403.csv")

In [ ]:
merge = pd.concat([df, master])

In [ ]:
merge = merge.drop("index", axis=1)

In [ ]:
merge = merge.reset_index()

In [ ]:
merge.shape

In [ ]:
#merge.to_csv("merged2560.csv")
merge = pd.read_csv("merged2560.csv")

In [ ]:
ne = df.copy()
nlp = spacy.load('en')

In [ ]:
unique = []
for i, row in ne.iterrows():
    try:
        doc = nlp(row.en)
        if doc.ents:
            #print('Name Entity: {0}'.format(doc.ents))
            for ent in doc.ents:
                print("phrase: ", row.en)
                print("NE: ", ent.text)
                my = str(input())
                if my == "*":
                    ne.loc[i,'en'] = ne.loc[i,'en'].replace(ent.text, "*")
                elif my == "":
                    continue
                else:
                    ne.loc[i,'en'] = ne.loc[i,'en'].replace(ent.text, my)
                    
                if ent.text not in unique:
                    unique.append(ent.text)
                    
    except TypeError:
        pass
    

In [ ]:
ne


## Database checklist

* \*BREAK* check
* Chinese named entity removal
* multiple definition check
* POS tag check
* 60 + dollars -> 60 dollars BUT dollars + 60 -> dollars w/o number

---

## Template options

* pronoun
* proper noun
* location
* time



In [ ]:
copy = ne.copy()
copy.sort_values(by='en', inplace=True)
copy.drop('Unnamed: 0', axis=1, inplace=True)
copy.head()

In [ ]:
master = pd.read_csv("master0403.csv")
#master.drop('ar', axis=1, inplace=True)
m1 = pd.concat([master, copy])

In [ ]:
#m1.drop("Unnamed: 0", axis=1, inplace=True)
m1.sort_values(by='en', inplace=True)
m1 = m1[['en', 'cn', 'ar', 'pos']]
m1.to_csv("m1.tsv", sep="\t")    

# Cleaned and merged 2500 phrases

In [ ]:
clean = pd.read_csv("clean2500-0410.csv")

In [ ]:
clean.shape

In [ ]:
clean.head()

In [ ]:
clean.ar.fillna("")

In [ ]:
clean[clean.duplicated(['en'], keep="first")]

In [ ]:
clean.loc[1551,'en'] = "to open"

In [ ]:
clean.drop(2472, inplace=True)

In [ ]:
for i, word in clean.iterrows():
    #if not str(i)[0].isalpha():
    if word.cn == ".你可以走了，带好你的东西。":
        print(i)

In [ ]:
clean.iloc[2441]

In [ ]:
clean.loc[2441,'cn']="你可以走了，带好你的东西。"
clean.loc[2441,'en']="You may go now. Please take your things."

In [ ]:
clean = clean.reset_index()
clean.drop('index', axis=1,inplace=True)

In [ ]:
clean.drop('level_0', axis=1,inplace=True)
clean.head()

In [ ]:
clean.sort_values('en')
clean.drop(2471, inplace=True)

In [ ]:
clean.to_csv("cleaned0410.csv")

In [ ]:
clean.drop([26], inplace=True)

In [ ]:
for i,row in clean.iterrows():
    if row.pos == "adv":
        print(row)

In [ ]:
clean.loc[2065, "en"] = "To *."
clean.loc[2065, "cn"] = "去*。"
clean.loc[2065, "ar"] = "إلى *."
clean.loc[2065, "pos"] = "vp"

In [ ]:
clean.loc[2065]

In [ ]:
clean.to_csv("cleaned0410.csv")

## Note: only vp and prt can contain *

In [ ]:
clean.head()

# Chinese segments

In [ ]:
df = pd.read_csv("cleaned0410.csv")

In [ ]:
import jieba

In [ ]:
for i, row in df.iterrows():
    df.loc[i, 'cn_split'] = " ".join(jieba.cut(str(row.cn)))

In [ ]:
df.head()
df.to_csv("cn_segmented.csv")

In [ ]:
df.head()

In [9]:
df = pd.read_csv("cleaned0410.csv")

In [10]:
df2 = pd.read_csv("addwords0417.csv")

In [13]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [14]:
df.head()

,en,cn,ar,pos
0,* a.m.,下午*点,* صباحًا,prt
1,* airlines has a flight leaving at *.,*航空公司有一架班机，在*起飞。,NaN,vp
2,"* and * , please.",请给我*和*。,من فضلك أعطني * و *.,vp
3,* day(s),*天,‏* أيام,prt
4,* dinar,第纳尔,دينار,prt


In [12]:
df2.head()

,en,cn,ar,pos
0,* and *,*和*,* و *,conj
1,Afghan,阿富汗,الأفغاني,adj
2,after all *,毕竟 *,بعد كل ذلك *,conj
3,although *,尽管*,برغم من *,conj
4,American,美国,أمريكي,adj


In [15]:
df = pd.concat([df, df2])

In [16]:
df.shape

(2595, 4)

In [17]:
df.tail(10)

,en,cn,ar,pos
115,until *,直到 *,حتى *,conj
116,Uruguayan,乌拉圭人,أوروجواي,adj
117,US,我们,لنا,adj
118,Venezuelan,委内瑞拉的,فنزويلا,adj
119,Vietnamese,越南的,الفيتنامية,adj
120,Welsh,威尔士语,تهرب من دفع الرهان,adj
121,wherever,哪里,أينما,conj
122,whether *,是否*,سواء *,conj
123,while *,而*,في حين *,conj
124,yet *,然而*,بعد *,conj


In [22]:
df[df.en.str.lower().duplicated(keep='first')]

,en,cn,ar,pos
1125,information,问讯处,NaN,np
1504,nothing to declare,无申报通道,NaN,np


In [32]:
lower = df.apply(lambda x: x.astype(str).str.strip(".,")).apply(lambda x: x.astype(str).str.lower())

In [35]:
dup = lower[lower.en.duplicated(keep='first')]

In [36]:
dup.index

Int64Index([  13,   18,   65,  112,  197,  239,  319,  435,  437,  462,  467,
             552,  555,  598, 1125, 1137, 1265, 1302, 1477, 1504, 1545, 1704,
            1782, 1871, 1876, 1893, 2152, 2162, 2164, 2166, 2216],
           dtype='int64')

In [37]:
for i in dup.index:
    df.drop(i, inplace=True)

In [38]:
df.shape

(2560, 4)

In [42]:
df.head(50)

,en,cn,ar,pos
0,* a.m.,下午*点,* صباحًا,prt
1,* airlines has a flight leaving at *.,*航空公司有一架班机，在*起飞。,NaN,vp
2,"* and * , please.",请给我*和*。,من فضلك أعطني * و *.,vp
3,* day(s),*天,‏* أيام,prt
4,* dinar,第纳尔,دينار,prt
5,* dirham,迪拉姆,درهم,prt
6,* dollar(s),美元,دولار,prt
7,* hour(s),*个小时,‏* ساعات,prt
8,* is not working.,坏了,* لا يعمل,vp
9,* minute(s),*分钟,‏* دقائق,prt


In [44]:
for i, row in df.iterrows():
    if "日语" in str(row.cn):
        print(row)

en     Do you speak *?
cn              你说日语吗?
ar                 NaN
pos                 vp
Name: 534, dtype: object


In [45]:
df.loc[534, 'cn'] = "你说*语吗?"
df.loc[534, 'ar'] = "هل تتكلم اللغة *؟"

In [46]:
df.loc[534]

en       Do you speak *?
cn                你说*语吗?
ar     هل تتكلم اللغة *؟
pos                   vp
Name: 534, dtype: object

In [47]:
df.to_csv("master0417.csv")

In [48]:
df.name

AttributeError: 'DataFrame' object has no attribute 'name'

In [51]:
df.columns[0]

'en'